In [1]:
import pandas as pd
import hashlib
import re
from cryptography.fernet import Fernet

In [10]:
file_path = 'Laptop_price.csv'
try:
    df = pd.read_csv(file_path)
except Exception as e:
    print(f"Ошибка загрузки файла: {e}")

In [11]:
def check_csv_injection(df):
    dangerous_chars = ('=', '+', '-', '@')
    results = []

    for col in df.select_dtypes(include=['object']).columns:
        pattern = r'\s*[' + ''.join(dangerous_chars) + r']'

        if df[col].str.contains(pattern).any():
            results.append(f"CSV-инъекции в столбце {col}")
        else:
            results.append(f"колонка {col} безопасен.")

    return results


check_csv_injection(df)

['колонка Brand безопасен.']

In [12]:
def clean_input(value):
    sql_keywords = ["SELECT", "DROP", "DELETE", "INSERT", "UPDATE", "ALTER", "UNION"]
    xss_patterns = [r'<script.*?>.*?</script>', r'javascript:.*', r'onerror.*']
    sql_comments_pattern = r'--'

    for keyword in sql_keywords:
        if keyword.lower() in value.lower():
            return "[BLOCKED]"

    if re.search(sql_comments_pattern, value):
        return "[BLOCKED]"

    for pattern in xss_patterns:
        if re.search(pattern, value, re.IGNORECASE):
            return "[BLOCKED]"

    return value

df = df.map(lambda x: clean_input(str(x)) if isinstance(x, str) else x)
print("Фильтрация данных завершена")

Фильтрация данных завершена


In [13]:
def hash_price(price):
    return hashlib.sha256(str(price).encode()).hexdigest()

df['Price_Hashed'] = df['Price'].apply(hash_price)
print("Столбец с хешированными ценами добавлен.")

key = Fernet.generate_key()
cipher = Fernet(key)

def encypt_price(price):
    return cipher.encrypt(str(price).encode()).decode()

def decrypt_price(encrypted_price):
    return cipher.decrypt(encrypted_price.encode()).decode()

def encrypt_ram(ram):
    return cipher.encrypt(str(ram).encode()).decode()

def decrypt_ram(encrypted_ram):
    return cipher.decrypt(encrypted_ram.encode()).decode()

df['Price_Encripted'] = df['Price'].apply(encypt_price)
print("Столбец с зашифрованными ценами добавлен.")

df['RAM_Size_Encrypted'] = df['RAM_Size'].apply(encrypt_ram)
print("Столбец с зашифрованной RAM добавлен.")

output_path = "Laptop_price_secured.csv"
df.to_csv(output_path, index=False)
print(f"Обработанный файл сохранен: {output_path}")
df

Столбец с хешированными ценами добавлен.
Столбец с зашифрованными ценами добавлен.
Столбец с зашифрованной RAM добавлен.
Обработанный файл сохранен: Laptop_price_secured.csv


,Brand,Processor_Speed,RAM_Size,Storage_Capacity,Screen_Size,Weight,Price,Price_Hashed,Price_Encripted,RAM_Size_Encrypted
0,Asus,3.830296,16,512,11.185147,2.641094,17395.093065,e33562234b73b4ecad16901885fd503f2c4c20996e22d9...,gAAAAABoHQgu1cC2chgik1SXyyZR4yNeFDaezgH0QqtZka...,gAAAAABoHQguAp-aFWPWh_p1i64Uc5iuhL7dcHhjrw0CX-...
1,Acer,2.912833,4,1000,11.311372,3.260012,31607.605919,80fe437d4a9c57c4038198b7cf32f93b9f957bb705b8e1...,gAAAAABoHQgu0VaErfq8WAXAGPkhSU_5dCTe9Ptw4G9heS...,gAAAAABoHQgu8hMsa_og4aBc_71IuvY6iH9SYBMysk53p-...
2,Lenovo,3.241627,4,256,11.853023,2.029061,9291.023542,6254a5a18d60d796eb464283ef38f9417e12885a07742e...,gAAAAABoHQguTsVwnHVwaTdgKtHHTyOh_0wmjw4thhFGJj...,gAAAAABoHQgu9gGqnj8PQ0F9AYVcD7E2k_HEVXAywLhSeo...
3,Acer,3.806248,16,512,12.280360,4.573865,17436.728334,1f2910923c33a53b30bc7e6eb731307eb2e2366b3811b6...,gAAAAABoHQgubohsu-m7Wt8zagkYErNlkMggKN2x2HR4yy...,gAAAAABoHQguBerUpW2UpCJb3nX-hx3l9Q370lpepZvUMI...
4,Acer,3.268097,32,1000,14.990877,4.193472,32917.990718,f90738a55c90c1e31a4fb6d35f92306ff03a87001a9408...,gAAAAABoHQgu4sD6JvW17-sVUtcQrcV3f1Cu6Q8n-D88e9...,gAAAAABoHQguaPwDkFl1rRmy5o-5cVf_b9qyEpq97avOso...
...,...,...,...,...,...,...,...,...,...,...
995,HP,3.343584,4,1000,12.587095,3.162399,31593.668017,17c0640d518f40efb4621a91128f37dd255a70325fb7d2...,gAAAAABoHQgu743M2VqVi1ZJj-7vn0YdxcUbf3vXkd7J_v...,gAAAAABoHQguBtQvDPpaZWItaJwjQ1i_A5H0gPVskieSl7...
996,Dell,2.780555,8,256,12.679356,3.750265,9149.521832,3c3a320eef285b3076573e44615fc5079327f9afcc65d1...,gAAAAABoHQguiksZZU9ZTfblD9q-Zj0XbrNXHdI7m3z3Hw...,gAAAAABoHQgunrID1YM-s-dc9Be8dGlFtPMAAxnbIL6c8n...
997,Dell,3.200569,4,512,12.666315,3.392612,16552.404779,fb06ef3c65b464a25611ccefec44f50e50e6d18a835738...,gAAAAABoHQgu71FLP5XACEaqnAUAUrHnRh57NS5oJsLxQP...,gAAAAABoHQgu1YtnNZI4ncqWoar4sjAt8vg-RxhGB-jeIl...
998,Asus,1.604182,8,256,11.215581,3.857613,9407.473459,dbe23db04d9d9d9537f6c5a65cb51b749612b1699857c0...,gAAAAABoHQgutNWhvWTOS-A3QpHqhojLRq0IinAW6zzUOi...,gAAAAABoHQguaclDQXcKVAHSsICJXmZyibiUE_fwcI-nzh...


In [14]:
decrypted_rams = df['RAM_Size_Encrypted'].apply(decrypt_ram).head(5)
print("5 расшифрованных значений RAM:")
print(decrypted_rams)

5 расшифрованных значений RAM:
0    16
1     4
2     4
3    16
4    32
Name: RAM_Size_Encrypted, dtype: object
